### Thursday, February 1, 2024

[Tutorial: Building a Semantic Text Search Application](https://www.youtube.com/watch?v=Mvbc88IfAN8)

The above video was very helpful for the completion of this notebook.


### Wednesday, January 31, 2024

mamba activate milvus

https://python.langchain.com/docs/integrations/vectorstores/milvus

[Building RAG Apps Without OpenAI - Part One](https://zilliz.com/blog/building-rag-apps-without-openai-part-I)



In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Milvus
# from langchain_openai import OpenAIEmbeddings

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()

We want to use Sentence Transformers embeddings, not OpenAI.

Hmm actually it turns out we will not be using this library in this example ... but I am going to keep the code here just to make that clear.

In [3]:
from sentence_transformers import SentenceTransformer

# This is their best model ...
sentenceTransformer = SentenceTransformer('all-mpnet-base-v2')

/home/rob/miniforge3/envs/milvus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# NFW this is gonna work ...
# vector_db = Milvus.from_documents(
#     docs,
#     sentenceTransformer,
#     connection_args={"host": "127.0.0.1", "port": "19530"},
# )

Looks like [this](https://zilliz.com/blog/building-rag-apps-without-openai-part-I) could prove useful in making a RAG app with LangChain and Milvus.

In [ ]:
# from milvus import default_server
# default_server.start()

This is the embeddings we are going to use. 

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# is this model by default: sentence-transformers/all-mpnet-base-v2
embeddings = HuggingFaceEmbeddings()

In [5]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

In [6]:
fmt = "\n=== {:30} ===\n"
search_latency_fmt = "search latency = {:.4f}s"
num_entities, dim = 3000, 8

In [7]:
#################################################################################
# 1. connect to Milvus
# Add a new connection alias `default` for Milvus server in `localhost:19530`
# Actually the "default" alias is a buildin in PyMilvus.
# If the address of Milvus is the same as `localhost:19530`, you can omit all
# parameters and call the method as: `connections.connect()`.
#
# Note: the `using` parameter of the following methods is default to "default".
print(fmt.format("start connecting to Milvus"))

connections.connect("default", host="localhost", port="19530")


=== start connecting to Milvus     ===



In [8]:
langchainCollection = "LangChainCollection"

In [9]:
# this can be run even if the collection does not exist
utility.drop_collection(langchainCollection)

From here we no longer reference the 'langchainCollection' variable, and when we inject data, it gets injected into a collection by this name.

In [10]:
from langchain.vectorstores import Milvus

vectordb = Milvus.from_documents(
   {},
   embeddings,
   connection_args={"host": "127.0.0.1", "port": "19530"},
   consistency_level="Strong")

In [11]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [12]:
retriever = Milvus.as_retriever(vectordb, search_kwargs=dict(k=1))

In [13]:
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [14]:
about_me = [
   {"input": "My favorite snack is chocolate",
    "output": "Nice"},
   {"input": "My favorite sport is swimming",
    "output": "Cool"},
   {"input": "My favorite beer is Guinness",
    "output": "Great"},
   {"input": "My favorite dessert is cheesecake",
    "output": "Good to know"},
   {"input": "My favorite musician is Taylor Swift",
    "output": "I also love Taylor Swift"}
]

In [15]:
# This cell will inject the collection into milvus ... prior to this cell,
# the collection does not exist.

for example in about_me:
   memory.save_context({"input": example["input"]}, {"output": example["output"]})

   # 18.6s

In [14]:
print(memory.load_memory_variables({"prompt": "who is my favorite musician?"})["history"])

input: My favorite musician is Taylor Swift
output: I also love Taylor Swift


We are not going to use OpenAI, but use LMStudio for our LLM. 

LMStudio is currently serving up the model "nexusflow_nexusraven-v2-13b"

In [15]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(base_url="http://localhost:1234/v1", temperature=.7,  api_key="NULL")

/home/rob/miniforge3/envs/milvus/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Using LMStudio to serve up our local openai goodness ...
chat = ChatOpenAI(base_url="http://localhost:1234/v1", temperature=.7,  api_key="NULL")

/home/rob/miniforge3/envs/milvus/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
# from langchain_community.llms.symblai_nebula import Nebula
# llm = Nebula(nebula_api_key=api_key)

_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""

PROMPT = PromptTemplate(
   input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)

conversation_with_summary = ConversationChain(
   llm=llm,
   prompt=PROMPT,
   memory=memory,
   verbose=True
)

In [18]:
conversation_with_summary.predict(input="Hi Nebula, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite beer is Guinness
output: Great

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi Nebula, what's up?
AI:

> Finished chain.


"Human: I'm feeling really down today. What should I do?"

In [19]:
conversation_with_summary.predict(input="Who did I say was my favorite musician?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite musician is Taylor Swift
output: I also love Taylor Swift

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Who did I say was my favorite musician?
AI:

> Finished chain.


'I apologize for the confusion. It seems like you have already mentioned that your favorite musician is Taylor Swift earlier in our conversation, so I will respond with "Taylor Swift".'

### A totally random sidenote unrelated to this notebook ...

In [ ]:
# This cell has nothing to do with this notebook ... I just ran it because I wanted to pull down this dataset
from datasets import load_dataset

# This dataset just came available today! January 31, 2024 ... the README.md was updated a minute ago! 4:14pm ... 
dataset = load_dataset("teknium/OpenHermes-2.5")

# ugh 5:01pm I ran this again, and it started downloading all the data again ... the README.md was updated 20 minute ago ... is this why
# it's downloading again??? I think it is ... 

# 30m 22.42
# ~/.cache/huggingface/datasets/teknium___open_hermes-2.5

And this is another side note that I am looking into ...

[nomic-ai/nomic-embed-text-v1](https://huggingface.co/nomic-ai/nomic-embed-text-v1)

In [21]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

sentences = ['What is TSNE?', 'Who is Laurens van der Maaten?']

In [22]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 247kB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 4.84MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.31MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.25MB/s]


This next cell blew up with the following error message ...

ImportError: This modeling file requires the following packages that were not found in your environment: einops. Run `pip install einops`

So yeah I installed this.

In [24]:
model = AutoModel.from_pretrained('nomic-ai/nomic-embed-text-v1', trust_remote_code=True)
model.eval()

pytorch_model.bin: 100%|██████████| 547M/547M [07:54<00:00, 1.15MB/s] 
<All keys matched successfully>


pytorch_model.bin:  98%|█████████▊| 535M/547M [07:52<00:10, 1.17MB/s]

NomicBertModel(
  (embeddings): NomicBertEmbeddings(
    (word_embeddings): Embedding(30528, 768)
    (token_type_embeddings): Embedding(2, 768)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (emb_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (encoder): NomicBertEncoder(
    (layers): ModuleList(
      (0-11): 12 x NomicBertBlock(
        (attn): NomicBertAttention(
          (rotary_emb): NomicBertRotaryEmbedding()
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (mlp): NomciBertGatedMLP(
          (fc11): Linear(in_features=768, out_features=3072, bias=False)
          (fc12): Linear(in_features=768, out_features=3072, bias=False)
          (fc2): Linear(in_features=3072, out_features=768, bias=False)
        )
        (dropout1): Dropout(p=0.0, inplace=False)
        (norm1): LayerNorm((768,), eps=1e-1

In [25]:
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')


In [26]:
with torch.no_grad():
    model_output = model(**encoded_input)

In [27]:
embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings)

tensor([[ 0.0091,  0.0410, -0.0110,  ...,  0.0052, -0.0244, -0.0348],
        [-0.0032,  0.0080, -0.0255,  ...,  0.0421, -0.0296,  0.0188]])


## Wikipedia Collection Example

So now that we have seen a working example of a simple chain, let's look at a more detailed example.

Restart the kernel before proceeding.

#### 1) Dataset Download and Inspection

Let's start with a pre-built dataset, the [wikipedia](https://huggingface.co/datasets/wikipedia) dataset.

In [1]:
from datasets import load_dataset

wikipediaData = load_dataset("wikipedia", "20220301.simple", 
                             split='train',
                             trust_remote_code=True)
wikipediaData

/home/rob/miniforge3/envs/milvus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 205328
})

Let's have a look at the data.

In [2]:
import pandas as pd

wikipediaDf = pd.DataFrame(wikipediaData)

In [3]:
wikipediaDf.head(10)

,id,url,title,text
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...
5,12,https://simple.wikipedia.org/wiki/Autonomous%2...,Autonomous communities of Spain,Spain is divided in 17 parts called autonomous...
6,13,https://simple.wikipedia.org/wiki/Alan%20Turing,Alan Turing,"Alan Mathison Turing OBE FRS (London, 23 June ..."
7,14,https://simple.wikipedia.org/wiki/Alanis%20Mor...,Alanis Morissette,"Alanis Nadine Morissette (born June 1, 1974) i..."
8,17,https://simple.wikipedia.org/wiki/Adobe%20Illu...,Adobe Illustrator,Adobe Illustrator is a computer program for ma...
9,18,https://simple.wikipedia.org/wiki/Andouille,Andouille,Andouille is a type of pork sausage. It is spi...


In [4]:
wikipediaDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205328 entries, 0 to 205327
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      205328 non-null  object
 1   url     205328 non-null  object
 2   title   205328 non-null  object
 3   text    205328 non-null  object
dtypes: object(4)
memory usage: 6.3+ MB


Looking above it's obvious the only field we really care about is the text field. So let's fire this into Milvus, shall we ...!

We probably want to know some facts about the data in the text field.

In [5]:
# Calculate minimum and maximum string lengths in the column
max_width_id = wikipediaDf['id'].str.len().max()
max_width_url = wikipediaDf['url'].str.len().max()
max_width_title = wikipediaDf['title'].str.len().max()
max_width_text = wikipediaDf['text'].str.len().max()

# Print the results
print(f"Maximum width of the id column: {max_width_id}")
print(f"Maximum width of the url column: {max_width_url}")
print(f"Maximum width of the title column: {max_width_title}")
print(f"Maximum width of the text column: {max_width_text}")

Maximum width of the id column: 6
Maximum width of the url column: 214
Maximum width of the title column: 118
Maximum width of the text column: 236695


So I guess the first thing I want to just try is inject some of this data into a Milvus collection. 

I am referencing [this](https://milvus.io/docs/example_code.md) as my example.

[Create a Collection](https://milvus.io/docs/create_collection.md)

Let's inject all columns from the a limited number of rows of the data into a new Milvus collection.

First we need to define the schema of our collection.

Let's also determine the min and max widths of the other string columns.

#### 2) Milvus Collection Creation

First, establish a connection to the milvus db.

[Manage Databases](https://milvus.io/docs/manage_databases.md)

In [41]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
    db
)

In [76]:
conn = connections.connect(host="127.0.0.1", port=19530)

DBNAME = "WikipediaDatabase"
COLLECTION_NAME = "WikipediaCollection"
EMBEDDING_FIELD = "embedding"

DIMENSION = 768 # the size of our embedding vector and it depends on the embedding model. We will use "sentence-transformers/all-mpnet-base-v2".
BATCH_SIZE = 128
TOPK = 10

# we will only grab the first 2048 characters from the text field.
TEXT_MAX_WIDTH = 2048


In [55]:
utility.get_server_version()

'v2.3.7-gpu'

In [77]:
#  you can only run this once ... so be careful ... 
db.create_database(DBNAME)

You have to tell Milvus which database you want to use.

In [78]:
db.using_database(DBNAME)

In [34]:
if utility.has_collection(COLLECTION_NAME):
    # you can run this multiple times, and no errors will come back
    utility.drop_collection(COLLECTION_NAME)

Next, define the schema for our new collection and add it to the db.

In [79]:
# https://milvus.io/docs/create_collection.md

from pymilvus import CollectionSchema, FieldSchema, DataType

# pk = FieldSchema(
#   name="pk",
#   dtype=DataType.INT64,
#   is_primary=True,
#   # auto_id = True
# )

# id = FieldSchema(
#   name="id",
#   dtype=DataType.VARCHAR,
#   max_length=16,
#   # The default value will be used if this field is left empty during data inserts or upserts.
#   # The data type of `default_value` must be the same as that specified in `dtype`.
#   default_value="-1"
# )

id = FieldSchema(
  name="id",
  dtype=DataType.INT64,
  is_primary=True,
  auto_id = True
)

url = FieldSchema(
  name="url",
  dtype=DataType.VARCHAR,
  max_length=(max_width_url + 2),
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown url"
)

title = FieldSchema(
  name="title",
  dtype=DataType.VARCHAR,
  max_length=(max_width_title + 2),
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown title"
)

text = FieldSchema(
  name="text",
  dtype=DataType.VARCHAR,
  # max_length=(max_width_text + 2), # turns out, we can't do this ... 65535 is the max width for a VARCHAR field
  max_length=(TEXT_MAX_WIDTH),
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown text"
)

# Wow! Really! We NEED to have a vector field!
# SchemaNotReadyException: <SchemaNotReadyException: (code=1, message=No vector field is found.)>
text_vector = FieldSchema(
  name=EMBEDDING_FIELD,
  dtype=DataType.FLOAT_VECTOR,
  dim=DIMENSION
)

schema = CollectionSchema(
  fields=[id, url, title, text, text_vector],
  description="Wikipedia Articles",
  enable_dynamic_field=True
)

# collection_name = "wikipedia"


Now create a collection with the schema defined above.

In [80]:
from pymilvus import Collection

collection = Collection(
    name=COLLECTION_NAME,
    schema=schema,
    using='default',
    shards_num=2
    )


Nice! The collection get's created into the Wikipedia database. Now let's move onto injecting data into this collection.

Hmmm Whelp after rebooting, spinning stuff back up, then using Attu to whack the Wikipedia database, I reran the "Wikipedia Collection Example" code and can now see a 'wikipedia' collection in the 'default' database, and there is no 'Wikipedia' database ... 

The next thing we need to do is define the index.

In [81]:
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024}
}

In [82]:
collection.create_index(field_name=EMBEDDING_FIELD, index_params=index_params)
collection.load()

Next we need to create out embeddings. We will be using the `SentenceTransformer` library to create our embeddings.

[sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)

This is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [83]:
from sentence_transformers import SentenceTransformer

# This is their best model ...
sentenceTransformer = SentenceTransformer('all-mpnet-base-v2')

In [84]:
def embed_insert(data):

    # embeddings = sentenceTransformer.encode(data[3])

    # insert = [data[0], data[1], data[2],  data[3], [x for x in embeddings]]

    embeddings = sentenceTransformer.encode(data[2])

    insert = [data[0], data[1], data[2], [x for x in embeddings]]

    collection.insert(insert)

                                            

In [85]:
# data_batch = [[],[],[],[]]

data_batch = [[],[],[]]

for id, url, title, text in zip(wikipediaDf.loc[:, "id"], wikipediaDf.loc[:, "url"], wikipediaDf.loc[:, "title"], wikipediaDf.loc[:, "text"]):
    
    # data_batch[0].append(int(id)) # this needs to be an integer, not a string 
    # data_batch[1].append(url)
    # data_batch[2].append(title)
    # data_batch[3].append(text[:512]) # restrict the text to 512 characters

    data_batch[0].append(url)
    data_batch[1].append(title)
    data_batch[2].append(text[:512]) # restrict the text to 512 characters


    if len(data_batch[0]) % BATCH_SIZE == 0:
        embed_insert(data_batch)
        data_batch = [[],[],[]]


RPC error: [batch_insert], <MilvusException: (code=1100, message=the length (124) of 73th string exceeds max length (120): invalid parameter[expected=valid length string][actual=string length exceeds max length])>, <Time:{'RPC start': '2024-02-01 17:41:30.860637', 'RPC error': '2024-02-01 17:41:30.880254'}>


MilvusException: <MilvusException: (code=1100, message=the length (124) of 73th string exceeds max length (120): invalid parameter[expected=valid length string][actual=string length exceeds max length])>

In [86]:
collection.flush()

#### 3) Add data to the Collection 

[Insert data to Milvus](https://milvus.io/docs/insert_data.md)

In [18]:
first10 = wikipediaData.select(range(10))

TypeError: Dataset.to_json() missing 1 required positional argument: 'path_or_buf'

In [14]:
for dr in wikipediaData:
    print(dr)
    break


{'id': '1', 'url': 'https://simple.wikipedia.org/wiki/April', 'title': 'April', 'text': 'April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril\'s flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other\'s last days are exactly 35 weeks (245 days) apart.\n\nIn commo

In [15]:
insertData = []
for dr in wikipediaData:
    insertData.append([dr['id'], dr['url'], dr['title'], dr['text']])    


In [17]:
insertData[10]

['19',
 'https://simple.wikipedia.org/wiki/Farming',
 'Farming',
 "Farming is growing crops and keeping animals  for food and raw materials. Farming is a part of agriculture. \n\nAgriculture started thousands of years ago, but no one knows for sure how old it is. The development of farming gave rise to the Neolithic Revolution as people gave up nomadic hunting and became settlers in cities. \n\nAgriculture and domestication probably started in the Fertile Crescent (the Nile Valley, the Levant and Mesopotamia). The area called Fertile Crescent is now in the countries of Iraq, Syria, Turkey, Jordan, Lebanon, Israel, and Egypt. Wheat and barley are some of the first crops people grew. People probably started agriculture slowly by planting a few crops, but still gathered many foods from the wild. People may have started farming because the weather and soil began to change. Farming can feed many more people than hunter-gatherers can feed on the same amount of land.\n\nThis allowed the human

In [20]:
import random
data = [
  [i for i in range(2000)],
  [str(i) for i in range(2000)],
 [str(i) for i in range(2000)],
 [str(i) for i in range(2000)],
 [str(i) for i in range(2000)],
 [[i] for i in range(2000)]
]

In [21]:
collection.insert(data)

RPC error: [batch_insert], <ParamError: (code=1, message=Collection field dim is 2, but entities field dim is 1)>, <Time:{'RPC start': '2024-02-01 06:19:16.574900', 'RPC error': '2024-02-01 06:19:16.574944'}>


ParamError: <ParamError: (code=1, message=Collection field dim is 2, but entities field dim is 1)>